In [ ]:
from google.colab import drive
import pandas as pd
import glob
import datetime
from dateutil.relativedelta import *
import re

In [ ]:
drive.mount('/content/drive')
%cd '/content/drive/Shared drives/C2ST_CommunityCorpsW21/DataAnalysis-SharedFolder'
# !ls  # list all files under 'DataAnalysis-SharedFolder' folder

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1qAC2LEAd_wBx08vbq4DzJhyCxfzsd6mq/Data Analysis- Shared Folder


# 1. programs_attendance_revised


#### 1.1. Read 'Programs Attendance revised 9.23.20.xlsx' file under Engagement Data folder into DataFrame 

In [ ]:
%cd '/content/drive/Shared drives/C2ST_CommunityCorpsW21/DataAnalysis-SharedFolder/Engagement Data/'
program_df = pd.read_excel('Programs Attendance revised 9.23.20.xlsx')

# Drop unneccessary rows
# Easiest way: program_df[:277] --> need to update slicing when new data comes in
program_df = program_df.dropna(subset=['Date'])
program_df = program_df[~program_df['Date'].str.contains('July',na=False)]
program_df = program_df[~program_df['Date'].str.contains('Total',na=False)]
program_df = program_df[~program_df['Date'].str.contains('Mean',na=False)]   

/content/drive/.shortcut-targets-by-id/1qAC2LEAd_wBx08vbq4DzJhyCxfzsd6mq/Data Analysis- Shared Folder/Engagement Data


In [ ]:
# Format Date of program 'Energy Colloquium': 5/5&7/08 -> 5/5/08, 5/7/08
program_df=program_df.loc[:,~program_df.columns.str.contains('^Unnamed', case=False)]
format_date = program_df[program_df['Date']=='5/5&7/08']    
format_date.at[4, 'Date'] = '5/7/08'
program_df = pd.concat([program_df, format_date], ignore_index=True)
program_df.at[4,'Date'] = '5/5/08'

In [ ]:
# Rename Program Title
program_df['Program Title']= program_df['Program Title'].str.strip()
program_df.at[program_df[program_df['Program Title'] == 'Women in STEM: Connect'].index[0],'Program Title']='2014 Women in STEM'
program_df.at[program_df[program_df['Program Title'] == '2015 Women in STEM: Connect'].index[0],'Program Title']='Women in STEM: Connect 2015'
program_df.at[program_df[program_df['Program Title'] == 'An Evening into the Multiverse'].index[0],'Program Title']='An Evening Inside the Multiverse'

In [ ]:
# Convert Date column to datetime
program_df['Date'] = pd.to_datetime(program_df['Date'])

In [ ]:
# Create program_id
program_df['Program ID'] = program_df.index
program_df['Program ID'] = 'Program_' + program_df['Program ID'].astype(str)
program_df.loc[program_df['Date'] == '2008-05-07', 'Program ID'] = 'Program_4'

In [ ]:
program_df        #[program_df['Program Title']=='Energy Colloquium']

,Date,Program Title,Live Stream,YouTube Views,FB Views,Attendance (in seats),Hours,Program Series,Venue,Type of Program,Male Speakers,Female Speakers,People of Color (informal),# of Speakers,Partner/ event,Total Viewing,Program ID
0,2007-10-05,Anniversary of Sputnik,NaN,NaN,NaN,150,1.5,PS,NU Thorne,Original,NaN,NaN,NaN,NaN,NaN,150,Program_0
1,2007-10-29,Nasa Administrative Griffin,NaN,NaN,NaN,131,1.5,PS,NU Thorne,Original,NaN,NaN,NaN,NaN,NaN,131,Program_1
2,2008-02-22,Stem Cell,NaN,NaN,NaN,100,1.5,SS,Co-Sponsor - All Day Event,Original,NaN,NaN,NaN,NaN,NaN,100,Program_2
3,2008-03-12,Future of Science: Rosner,NaN,NaN,NaN,50,1.5,SS,Cultural Center,Original,1,0,0,1,NaN,50,Program_3
4,2008-05-05,Energy Colloquium,NaN,NaN,NaN,60,1.5,CEE,Exelon/Co-Sponsor,Original,NaN,NaN,NaN,NaN,NaN,60,Program_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,2020-08-27,Lessons in Biology Taught by a Pathogen,107,76,31.0,NaN,1,LS,Facebook Live & YouTube,Original,1,0,0,1,Chitown Bio,107,Program_273
274,2020-09-01,UIC Involvement Fair,24,0,0.0,NaN,NaN,SS,Booth,Original,0,2,NaN,NaN,NaN,NaN,Program_274
275,2020-09-24,The Power of Genomics: Optimzing Health Outcom...,88,0,88.0,NaN,1,LS,Facebook Live & YouTube,Original,1,0,0,1,IGB; Kleinmuntz Center,NaN,Program_275
276,2020-10-12,Deploying SHIELD: To Illinois and Beyond,125,43,82.0,NaN,1,HW,Facebook Live & YouTube,Original,2,0,0,2,University of Illinois System,NaN,Program_276


In [ ]:
# program_df      #uncomment to show program_df

# 2. data_studio_monthly_youtube

### 2.1 Youtube monthly views

In [ ]:
# Step1
path =r'/content/drive/Shared drives/C2ST_CommunityCorpsW21/TestFiles/Youtube_monthly/Youtube_views'
youtube_view_files = glob.glob(path + "/*.csv")

# 1.Loop through csv files under Youtube_views folder 
# 1) read them into DataFrames 
# 2) create column: Update Time 
youtube_view_dataframes = []
view_start_date = datetime.date(2020, 2, 1)

max_len = 0
for file in youtube_view_files:
  if len(file) > max_len: max_len = len(file) 

for file in youtube_view_files:
  if len(file) == max_len:
    year = int(file[-14:-10])
    month = int(file[-9:-7])
    date = int(file[-6:-4])
    view_start_date = datetime.date(year, month, date)
  else:
    year = int(file[-13:-9])
    month = int(file[-8:-7])
    date = int(file[-6:-4])
    view_start_date = datetime.date(year, month, date)
  view_df = pd.read_csv(file)
  view_df['Update Date'] = view_start_date
  youtube_view_dataframes.append(view_df)

# Concatenate all data into one DataFrame
youtube_view_df = pd.concat(youtube_view_dataframes)
youtube_view_df = youtube_view_df.drop(columns=['% Δ'])
# youtube_view_df

### 2.2 Youtube monthly watch times

In [ ]:
path =r'/content/drive/Shared drives/C2ST_CommunityCorpsW21/TestFiles/Youtube_monthly/Youtube_watchtimes' ## Directory needs to change
youtube_length_files = glob.glob(path + "/*.csv")

# Loop through csv files under Youtube_watchtimes folder
# 1) read them into DataFrames 
# 2) create column: Update Time 
youtube_length_dataframes = []
length_start_date = datetime.date(2020, 2,1)

max_len2 = 0
for file in youtube_length_files:
  if len(file) > max_len2: max_len2 = len(file) 

for file in youtube_length_files:
  if len(file) == max_len2:
    year = int(file[-14:-10])
    month = int(file[-9:-7])
    date = int(file[-6:-4])
    length_start_date = datetime.date(year, month, date)
  else:
    year = int(file[-13:-9])
    month = int(file[-8:-7])
    date = int(file[-6:-4])
    length_start_date = datetime.date(year, month, date)
  length_df = pd.read_csv(file)
  length_df['Update Date'] = length_start_date
  youtube_length_dataframes.append(length_df)

# Concatenate all data into one DataFrame
youtube_length_df = pd.concat(youtube_length_dataframes, ignore_index=True)
youtube_length_df = youtube_length_df.drop(columns=['% Δ'])
#youtube_length_df

### 2.3 Merge youtube_view_df and youtube_length_df based on video title and update date

In [ ]:
# 1. Merge youtube_view_df and youtube_length_df on 'Video Tile' and 'Update Date'
youtube_studio_df = pd.merge(youtube_view_df,youtube_length_df,how="outer", on=["Video Title", "Update Date"])
youtube_studio_df['Update Date'] = pd.to_datetime(youtube_studio_df['Update Date'])

# 2. Create column 'Program ID' in youtube_studio_df: 
youtube_studio_df['Program ID'] = youtube_studio_df.index    
youtube_studio_df['Program ID'] = 'Youtube_' + youtube_studio_df['Program ID'].astype(str)

# 3. Replace Program ID for the same video: 
# e.g. Video Title                  Update Date Program ID   -----> Video Title                  Update Date Program ID
    #  Women in STEM: Connect 2015	2020-02-01	379                 Women in STEM: Connect 2015	 2020-02-01	 379
    #  Women in STEM: Connect 2015  2020-09-01  1569                Women in STEM: Connect 2015  2020-09-01  379
for title in youtube_studio_df['Video Title']:
  for title2 in youtube_studio_df['Video Title']:
    if title2 == title:
      id = youtube_studio_df.loc[youtube_studio_df['Video Title'] == title]['Program ID'].values[0]
      youtube_studio_df.loc[youtube_studio_df['Video Title'] == title2, 'Program ID'] = id

In [ ]:
# 4. rename video title
youtube_studio_df.loc[youtube_studio_df['Video Title'] == 'Origins of Genus Homo', 'Video Title'] = 'Origins of the Genus Homo'
youtube_studio_df.loc[youtube_studio_df['Video Title'] == 'The Secret Lives of Female Chimpanzees', 'Video Title'] = 'The Secret Life of Female Chimpanzees'

In [ ]:
# 5. Add Youtube Publish Date
%cd '/content/drive/Shared drives/C2ST_CommunityCorpsW21/DataAnalysis-SharedFolder/YouTube'
youtube_df = pd.read_csv('Table data.csv')
youtube_df = youtube_df[['Video','Video title','Video publish time']]
youtube_df = youtube_df.dropna(subset=['Video title'])
youtube_df.loc[youtube_df['Video title'] == 'Origins of Genus Homo', 'Video Title'] = 'Origins of the Genus Homo'
youtube_df.loc[youtube_df['Video title'] == 'The Secret Lives of Female Chimpanzees', 'Video Title'] = 'The Secret Life of Female Chimpanzees'

for video in youtube_studio_df['Video Title']:
  for video2 in youtube_df['Video title']:
    if video == video2:
      date = youtube_df.loc[youtube_df['Video title'] == video2]['Video publish time'].values[0]
      youtube_studio_df.loc[youtube_studio_df['Video Title'] == video, 'Publish Date'] = date
youtube_studio_df['Publish Date'] = pd.to_datetime(youtube_studio_df['Publish Date'])

/content/drive/.shortcut-targets-by-id/1qAC2LEAd_wBx08vbq4DzJhyCxfzsd6mq/Data Analysis- Shared Folder/YouTube


In [ ]:
youtube_studio_df

,Video Title,Views,Update Date,Video Length,Watch Time,Program ID,Publish Date
0,The Origins of Genus Homo,473,2021-03-25,4661,1309.061815,Youtube_0,2018-06-18
1,COVID Vaccines: Fact vs. Fiction,221,2021-03-25,2897,1422.211903,Youtube_1,NaT
2,Biomechanics of Running: The Science of Moveme...,153,2021-03-25,4858,748.117743,Youtube_2,2014-09-13
3,Mental Help: Coping with Bias & Collective Trauma,124,2021-03-25,3673,1146.804989,Youtube_3,NaT
4,"Dietary Supplements: Stories of Regulation, Sa...",88,2021-03-25,5036,288.348500,Youtube_4,2017-06-26
...,...,...,...,...,...,...,...
2518,Climate Disruption: What We Can Do Now,1,2020-01-31,4845,59.935000,Youtube_300,2016-03-10
2519,The Science of Star Wars Highlights,1,2020-01-31,155,55.328500,Youtube_154,2019-04-11
2520,Music - There's a Science to That Highlights -...,1,2020-01-31,271,3.132000,Youtube_1930,2015-04-20
2521,"C2ST and 2Blades Present: Food Evolution, High...",0,2020-01-31,344,0.000000,Youtube_82,2019-01-07


## 2.4 Assign program_id to youtube video


### 2.4.1 Check how many programs use different program title and video title

In [ ]:
program_title = []
for title in program_df['Program Title']:
  program_title.append(title)

different_title = []
youtube_title = []
for title in youtube_studio_df['Video Title']:
  youtube_title.append(title)
  if title not in program_title:
    different_title.append(title)

print(len(set(different_title))) 
print(len(set(youtube_title)))    # 275 Unique Youtube Video Title

241
275


#### 2.4.2 Assign program_id to youtube video

In [ ]:
# Step1: Create a list that contains the words for video title identification
title_identification = []
for title in program_df['Program Title']:
  pattern = re.compile(r'[:(!]')
  match = pattern.search(title)
  if match:
    if pattern.split(title)[0] != 'SFTF':
      title_identification.append(pattern.split(title)[0].strip().lower()) 
      title_identification.append(pattern.split(title)[1].strip().lower())
    else:
      title_identification.append(pattern.split(title)[1].strip().lower())
  else:
    title_identification.append(title.lower())

for title in title_identification:
  if title == "":
    title_identification.remove(title)

In [ ]:
# Step2: Assign the program id to youtube video
for video in youtube_studio_df['Video Title']:
  # 2.1: For program that has same program title and youtube video title
  for program in program_df['Program Title']: 
    if program.lower() == video.lower():   
      id = program_df.loc[program_df['Program Title'] == program]['Program ID'].values[0]
      youtube_studio_df.loc[youtube_studio_df['Video Title'] == video, 'Program ID'] = id
  # 2.2: For program that has different title
  for word in title_identification:
    if word in video.lower() or video.lower() in word:
      for program in program_df['Program Title']:
        if word in program.lower():
          id = program_df.loc[program_df['Program Title'] == program]['Program ID'].values[0]
          youtube_studio_df.loc[youtube_studio_df['Video Title'] == video, 'Program ID'] = id

In [ ]:
# Step3: replace program id, eg.
# Video Title              Update Date        Program ID  ---> Video Title              Update Date        Program ID
# Science is for Everyone   2020-02-01         300             Science is for Everyone   2020-02-01         300
# Science is for Everyone   2020-03-01         400            Science is for Everyone   2020-03-01          300
for video in youtube_studio_df['Video Title']:
  for video2 in youtube_studio_df['Video Title']:
    if video2 == video:
      id = youtube_studio_df.loc[youtube_studio_df['Video Title'] == video]['Program ID'].values[0]
      youtube_studio_df.loc[youtube_studio_df['Video Title'] == video2, 'Program ID'] = id


In [ ]:
# Step4: check code
# youtube_studio_df[youtube_studio_df['Program ID']>277][20:50]
# for program in program_df['Program Title']:
#   for title in youtube_studio_df['Video Title']:
#     if program == title:
#       print(title)
# # id_list = []
# # for i in youtube_studio_df['Program ID']:
# #   id_list.append(i)
# # print(len(set(id_list)))     

# # title_different = []
# # for t in youtube_studio_df['Video Title']:
# #   title_different.append(t)

# # for i in set(title_different):
# #   print(i)

In [ ]:
# check code
# program_df[program_df['Program Title'].str.contains('Women in STEM')]
# youtube_studio_df[youtube_studio_df['Video Title'].str.contains('Women in STEM')]

# 3. facebook Table

#### 3.1 Read files into DateFrame and filter columns

In [ ]:
path =r'/content/drive/Shared drives/C2ST_CommunityCorpsW21/DataAnalysis-SharedFolder/Facebook'
facebook_files = glob.glob(path + "/*.csv")

# Loop through csv files under Facebook folder
facebook_dataframes = []
for file in facebook_files:
  facebook_dataframes.append(pd.read_csv(file))

# Concatenate all data into one DataFrame
facebook_df = pd.concat(facebook_dataframes, ignore_index=True)

In [ ]:
# Filter columns and create column Program ID
facebook_df = facebook_df.dropna(subset=['Post ID'])
facebook_df['publish_date'] = pd.to_datetime(facebook_df['Posted']).dt.date
facebook_df['publish_time'] = pd.to_datetime(facebook_df['Posted']).dt.time
facebook_df['Post Message'] = facebook_df['Post Message'].astype('str') 
facebook_df = facebook_df[['Post ID','Permalink','Post Message','Type','publish_date','publish_time','Lifetime Post Total Impressions','Lifetime Total Video Views']]

facebook_df['Program ID'] = facebook_df.index
facebook_df['Program ID'] = 'Facebook_' + facebook_df['Program ID'].astype(str)

In [ ]:
# Rewrite Post Message to include Program Title
# 6_15_2020-present.csv
facebook_df.loc[facebook_df['Post ID']=='163815626997995_2351117388522924','Post Message'] = 'An Evening Inside the Multiverse '+ facebook_df.loc[facebook_df['Post ID']=='163815626997995_2351117388522924','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_955219121616822','Post Message'] = '3D Printing in Action: An Introduction to Digital Fabrication '+ facebook_df.loc[facebook_df['Post ID']=='163815626997995_955219121616822','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_315939189682664','Post Message'] = 'Lessons in Biology Taught by a Pathogen ' +facebook_df.loc[facebook_df['Post ID']=='163815626997995_315939189682664','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_341645600563706','Post Message'] = 'Mindfully Coping: Strategies for Children and Families During a Pandemic ' +facebook_df.loc[facebook_df['Post ID']=='163815626997995_341645600563706','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_604032953849979','Post Message'] = 'SFTF: Engineering at the Service of Humanity '+facebook_df.loc[facebook_df['Post ID']=='163815626997995_604032953849979','Post Message'] 
facebook_df.loc[facebook_df['Post ID']=='163815626997995_2700080293429895','Post Message'] = 'Art, Science, and the Elegant Universe '+facebook_df.loc[facebook_df['Post ID']=='163815626997995_2700080293429895','Post Message']
# 12_2019.csv
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1691772297631304','Post Message'] = 'The Case for Including Every(Body) '+facebook_df.loc[facebook_df['Post ID']=='163815626997995_1691772297631304','Post Message']
# 12_2018.csv
facebook_df.loc[facebook_df['Post ID']=='163815626997995_554180258387399','Post Message'] = 'Bionics, Robotics, and the Future of Rehabilitation '+facebook_df.loc[facebook_df['Post ID']=='163815626997995_554180258387399','Post Message']
# 6_15_2019-11_2019
facebook_df.loc[facebook_df['Post ID']=='163815626997995_442777316441180','Post Message'] = 'The Art of Logic '+facebook_df.loc[facebook_df['Post ID']=='163815626997995_442777316441180','Post Message']
# 6_15_2018-11_2018.csv
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1914590188843520','Post Message'] = 'Sleep and Clocks: Windows to Health '+facebook_df.loc[facebook_df['Post ID']=='163815626997995_1914590188843520','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_300668803865600','Post Message'] = 'Head Banging: How Music and Concussions Affect Brain Health' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_300668803865600','Post Message']
# 6_15_2017-11_2017
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1497356340310577','Post Message'] = 'New Developments in Mental Health ' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_1497356340310577','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1541879275858283','Post Message'] = 'Math-It\'s as easy as Pi! ' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_1541879275858283','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1537280886318122','Post Message'] = 'Unconscious Bias '+ facebook_df.loc[facebook_df['Post ID']=='163815626997995_1537280886318122','Post Message'] 
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1539933329386211','Post Message'] ='Unconscious Bias ' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_1539933329386211','Post Message']
# 1_2020-6_15_2020.csv
facebook_df.loc[facebook_df['Post ID']=='163815626997995_577273266243327','Post Message'] = 'Science from the Frontlines: Racial Disparity in COVID-19 Panel ' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_577273266243327','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_586122298677190','Post Message'] = 'Science from the Frontlines:Gender Disparity in COVID-19 Cases ' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_586122298677190','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_252822859161206','Post Message'] = 'Sanitizer and Spirits ' +facebook_df.loc[facebook_df['Post ID']=='163815626997995_252822859161206','Post Message']
# 1_2019-6_15_2019.csv
facebook_df.loc[facebook_df['Post ID']=='163815626997995_307786086550940','Post Message'] ='The Science of Star Wars ' +facebook_df.loc[facebook_df['Post ID']=='163815626997995_307786086550940','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_445879205974812','Post Message'] = 'Lurie Prize Winner Lijian James Chen ' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_445879205974812','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1039672012891508','Post Message'] = 'The Science of Star Wars ' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_1039672012891508','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1237015259788586','Post Message'] = 'The Science of Star Wars ' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_1237015259788586','Post Message']
# 1_1_2018-6_15_2018.csv
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1748797515166457','Post Message'] =  'Extreme Storms ' +facebook_df.loc[facebook_df['Post ID']=='163815626997995_1748797515166457','Post Message']
facebook_df.loc[facebook_df['Post ID']=='163815626997995_1687463007966575','Post Message'] =  'The Future of Trees: Hope in a Changing Climate ' + facebook_df.loc[facebook_df['Post ID']=='163815626997995_1687463007966575','Post Message']
# 12_2017.csv 12_2016.csv 12_2015.csv 6_15_2016-11_2016.csv 1_1_2015-6_15_2015.csv 1_1_2017- 6_15_2017.csv: no needs for modification
# 6_15_2015-11_2015, 1_1_2016-6_15_2016.csv: Artist in Residence --> can't find any program relevant to this series


#### 3.2 Assign program id to facebook post

In [ ]:
for message in facebook_df['Post Message']:
  for word in title_identification:
    if word in message.lower():
      for program in program_df['Program Title']:
        if word in program.lower():
          id = program_df.loc[program_df['Program Title'] == program]['Program ID'].values[0]
          facebook_df.loc[facebook_df['Post Message'] == message, 'Program ID'] = id

# facebook_df[facebook_df['Permalink']=='https://www.facebook.com/C2ST1/videos/987053601340856/']   # uncomment to check whether assign the right program id

In [ ]:
# facebook_df

# 4. eventbrite_web_traffic

#### No data before 2020/2/1

In [ ]:
path =r'/content/drive/Shared drives/C2ST_CommunityCorpsW21/TestFiles/Eventbrite_web_traffic'
eventbrite_web_files = glob.glob(path + "/*.csv")

# 1.Loop through csv files under Youtube_views folder 
# 1) read them into DataFrames 
# 2) Create column: Update Time 
eventbrite_web_dataframes = []
eventweb_start_date = datetime.date(2020, 3, 1)

max_len3 = 0
for file in eventbrite_web_files:
  if len(file) > max_len3: max_len3 = len(file) 

for file in eventbrite_web_files:
  if len(file) == max_len3:
    year = int(file[-14:-10])
    month = int(file[-9:-7])
    date = int(file[-6:-4])
    eventweb_start_date = datetime.date(year, month, date)
  else:
    year = int(file[-13:-9])
    month = int(file[-8:-7])
    date = int(file[-6:-4])
    eventweb_start_date = datetime.date(year, month, date)
  event_df = pd.read_csv(file)
  event_df['Update Time'] = eventweb_start_date
  eventbrite_web_dataframes.append(event_df)

# Concatenate all data into one DataFrame
eventbrite_web_df = pd.concat(eventbrite_web_dataframes, ignore_index=True)
eventbrite_web_df = eventbrite_web_df.drop(columns=['% Δ'])


# Create Program ID and replace program id for same events(see example in 2.4.2 example)
eventbrite_web_df['Program ID'] = eventbrite_web_df.index
eventbrite_web_df['Program ID'] = 'Eventbrite_' + eventbrite_web_df['Program ID'].astype(str)
for event in eventbrite_web_df['Page Title']:
  for event2 in eventbrite_web_df['Page Title']:
    if event2 == event:
      id = eventbrite_web_df.loc[eventbrite_web_df['Page Title'] == event]['Program ID'].values[0]
      eventbrite_web_df.loc[eventbrite_web_df['Page Title'] == event2, 'Program ID'] = id

In [ ]:
for page in eventbrite_web_df['Page Title']:
  for word in title_identification:
    if word in page.lower():
      for program in program_df['Program Title']:
        if word in program.lower():
          id = program_df.loc[program_df['Program Title'] == program]['Program ID'].values[0]
          eventbrite_web_df.loc[eventbrite_web_df['Page Title'] == page, 'Program ID'] = id



In [ ]:
eventbrite_web_df

,Page Title,Pageviews,Update Time,Program ID
0,Many Pathways - Workshop and Network Event Tic...,329,2020-02-29,Program_254
1,"Art, Science, and the Elegant Universe Tickets...",212,2020-02-29,Program_265
2,"Eventbrite - Art, Science, and the Elegant Uni...",39,2020-02-29,Program_265
3,"You're going to Art, Science, and the Elegant ...",32,2020-02-29,Program_265
4,Eventbrite - Many Pathways - Workshop and Netw...,31,2020-02-29,Program_254
...,...,...,...,...
309,"Science and Sandwiches featuring Eric Gorscak,...",1,2021-03-25,Eventbrite_235
310,Mental Help: Coping with Bias & Collective Tra...,1,2021-03-25,Eventbrite_269
311,Eventbrite - A Window Into Science 2021,1,2021-03-25,Eventbrite_311
312,"A Window Into Science 2021 Tickets, Thu, Apr 1...",1,2021-03-25,Eventbrite_312


# 5. survey_reports

In [ ]:
%cd '/content/drive/Shared drives/C2ST_CommunityCorpsW21/TestFiles/Program_report'


/content/drive/Shared drives/C2ST_CommunityCorpsW21/TestFiles/Program_report


In [ ]:
path =r'/content/drive/Shared drives/C2ST_CommunityCorpsW21/TestFiles/Program_report'
survey_files = glob.glob(path + "/*.xlsx")

# Loop through csv files under Program Report folder and read them into DataFrame
survey_dataframes = []
for file in survey_files:
  try: 
    sheet2 = pd.read_excel(file,sheet_name =1)
    sheet1 = pd.read_excel(file,sheet_name =0)
    sheet = pd.concat([shee2,sheet1],ignore_index=True)
  except:
    sheet = pd.read_excel(file)
  survey_dataframes.append(sheet)

# Concatenate all data into one DataFrame and Transpose index and columns
survey_df = pd.concat(survey_dataframes, ignore_index=True)
survey_df = survey_df.loc[:,~survey_df.columns.str.contains('^Unnamed', case=False)]
survey_df = survey_df.set_index('Why did you attend this event?').T
survey_df = survey_df.reset_index().rename(columns={'index':'Program Title'})


In [ ]:
# Create column Program ID 
survey_df['Program ID'] = survey_df.index
survey_df['Program ID'] = 'Survey_' + survey_df['Program ID'].astype(str)
for survey in survey_df['Program Title']:
  for program in program_df['Program Title']:
    if survey.lower() == program.lower():
      id = program_df.loc[program_df['Program Title'] == program]['Program ID'].values[0]
      survey_df.loc[survey_df['Program Title'] == survey, 'Program ID'] = id
  
  for word in title_identification:
      if word in survey.lower() or survey.lower() in word:
        for program in program_df['Program Title']:
          if word in program.lower():
            id2 = program_df.loc[program_df['Program Title'] == program]['Program ID'].values[0]
            survey_df.loc[survey_df['Program Title'] == survey, 'Program ID'] = id2

In [ ]:
# survey_df

# 6. Merge all Dataframes

In [ ]:
# Step1: Merge program_df and youtube_studio_df
new_df = pd.merge(program_df,youtube_studio_df,how="outer", on=["Program ID"])
# Step2: Merge new_df and facebook_df
new_df = pd.merge(new_df,facebook_df,how="outer", on=["Program ID"]) 
# Step3: Merge new_df and eventbrite_web_df
new_df = pd.merge(new_df,eventbrite_web_df,how="outer", on=["Program ID"]) 
# Step4: Merge new_df and survey_df
new_df = pd.merge(new_df,survey_df,how="outer", on=["Program ID"]) 

new_df

,Date,Program Title_x,Live Stream,YouTube Views,FB Views,Attendance (in seats),Hours,Program Series,Venue_x,Type of Program,Male Speakers,Female Speakers,People of Color (informal),# of Speakers,Partner/ event,Total Viewing,Program ID,Video Title,Views,Update Date,Video Length,Watch Time,Publish Date,Post ID,Permalink,Post Message,Type,publish_date,publish_time,Lifetime Post Total Impressions,Lifetime Total Video Views,Page Title,Pageviews,Update Time,Program Title_y,Interest in topic,Interest in speaker,Venue_y,Schedule allowed for it,Networking,...,promotional materials,eventbrite,NaN,What is your gender?,Female,Male,Other,prefer not to say,age?,18 and under,19-24,25-34,35-44,45-54,55-64,65-74,75 and older,Prefer not to answer,what is your ethnicity?,american indian or alaskan native,asian or pacific islander,black or african american,hispanic or latino,middle eastern/north african,hawaiian/pacific islander,white or caucasian,other,prefer not to say,household income,"$0-$24,999","$25,000-$49,999","$50,000-$74,999","$75,000-$99,999","$100,000-$124,999","$125,000-$149,999","$150,000-174,999","$175,000-$199,999","$200,000+",Prefer not to say,NaN
0,2007-10-05,Anniversary of Sputnik,NaN,NaN,NaN,150,1.5,PS,NU Thorne,Original,NaN,NaN,NaN,NaN,NaN,150,Program_0,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-10-29,Nasa Administrative Griffin,NaN,NaN,NaN,131,1.5,PS,NU Thorne,Original,NaN,NaN,NaN,NaN,NaN,131,Program_1,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-02-22,Stem Cell,NaN,NaN,NaN,100,1.5,SS,Co-Sponsor - All Day Event,Original,NaN,NaN,NaN,NaN,NaN,100,Program_2,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-03-12,Future of Science: Rosner,NaN,NaN,NaN,50,1.5,SS,Cultural Center,Original,1,0,0,1,NaN,50,Program_3,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A Window into Science 2021 Future of Science t...,2.0,2021-03-25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-05-05,Energy Colloquium,NaN,NaN,NaN,60,1.5,CEE,Exelon/Co-Sponsor,Original,NaN,NaN,NaN,NaN,NaN,60,Program_4,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4961,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Survey_67,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Future of Trees.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4962,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Survey_68,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Science Riot 1.2.1,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,NaN,NaN,6.0,6.0,0.0,0.0,NaN,0.0,1.0,4.0,1.0,2.0,0.0,4.0,0.0